### Run basic `scanpy` QC and doublet detection with `scrublet` for **PBMC Tuberculosis** _Cai Y et al 2022_

**Objective**: Review QC process and suggest changes

- **Developed by**: Carlos Talavera-López PhD
- **Modified by**: Mairi McClean
- **Computational Health Centre - Helmholtz Munich**
- ORIGINAL: v221015; MODIFIED: v221116

### Load required modules

In [ ]:
import anndata
import logging
import numpy as np
import pandas as pd
import scanpy as sc
import seaborn as sb
import scrublet as scr
import matplotlib.pyplot as plt
from matplotlib import colors
from matplotlib import rcParams

In [ ]:
sc.settings.verbosity = 3
sc.logging.print_versions()
sc.settings.set_figure_params(dpi = 160, color_map = 'RdPu', dpi_save = 180, vector_friendly = True, format = 'svg')

### Read datasets

In [ ]:
adata = sc.read_h5ad('/Users/mairi.mcclean/github/data/tb_pbmc_datasets/CaiY2022_TB.raw.h5ad')
adata

In [ ]:
adata.var

In [ ]:
adata.obs

In [ ]:
adata.obs['donor'].value_counts()

In [ ]:
adata.obs['data_type'].values

In [ ]:
adata.var_names_make_unique()
sample_object = adata.copy()
sample_object

### Replace gene symbols 

In [ ]:
sample_object.var['gene_id'] = sample_object.var.index.copy()
sample_object.var.set_index('gene_name', inplace = True)
sample_object.var.head()

In [ ]:
sample_object.var_names = [str(i) for i in sample_object.var_names]
sample_object.var_names_make_unique()


### Inital scatterplot of top 20

In [ ]:
# highest fraction of counts per cell

sc.pl.highest_expr_genes(sample_object, n_top=20)

### Filter cells with less than 200 genes

In [ ]:
sc.pp.filter_cells(sample_object, min_genes = 200)
print(sample_object.n_obs, sample_object.n_vars)

In [ ]:
sample_object.shape

In [ ]:
sample_object.var

### Remove all data that is not scRNAseq

In [ ]:
sample_object.obs

In [ ]:
sample_object.obs['data_type']

In [ ]:
sample_object.obs['data_type'].cat.categories

In [ ]:
# Code from https://scanpy.discourse.group/t/filter-out-specific-clusters-using-their-cluster-number/82

sample_object_new = sample_object[~sample_object.obs['data_type'].isin(['scTCRseq']),:]

In [ ]:
sample_object_new

In [ ]:
sample_object_new.obs['data_type'].cat.categories

### Compute QC stats

In [ ]:
sample_object_new.shape

In [ ]:
sample_object.var['mt'] = sample_object.var_names.str.startswith('MT-')
sample_object.var['ribo'] = sample_object.var_names.str.startswith(("RPS","RPL"))
sample_object.var

In [ ]:
sc.pp.calculate_qc_metrics(sample_object, qc_vars = ['mt', 'ribo'], percent_top = None, log1p = False, inplace = True)

In [ ]:
# for each cell compute fraction of counts in mito genes vs. all genes
# the `.A1` is only necessary as X is sparse (to transform to a dense array after summing)
# add the total counts per cell as observations-annotation to adata

mito_genes = sample_object_new.var_names.str.startswith('MT-')
sample_object_new.obs['percent_mt2'] = np.sum(
    sample_object_new[:, mito_genes].X, axis = 1).A1 / np.sum(sample_object_new.X, axis = 1).A1
sample_object_new.obs['n_counts'] = sample_object_new.X.sum(axis = 1).A1

In [ ]:
sample_object_new 

### Visualise QC metrics

In [ ]:
sample_object_new.var_names

In [ ]:
# This particular visualisation was from Anna's notebook.

sc.pl.violin(sample_object_new, ['n_genes_by_counts', 'total_counts', 'pct_counts_mt', 'pct_counts_ribo'],
             jitter=0.4, multi_panel=True)

In [ ]:
sc.pl.violin(sample_object_new, ['n_genes_by_counts', 'total_counts', 'pct_counts_mt','pct_counts_ribo'],
             jitter = 0.2, groupby = 'donor', rotation = 45)

In [ ]:
sc.pl.violin(sample_object_new, ['n_genes_by_counts', 'total_counts', 'pct_counts_mt','pct_counts_ribo'],
             jitter=0.4, groupby = 'tissue', rotation = 45)

In [ ]:
sc.pl.scatter(sample_object_new, x = 'total_counts', y = 'pct_counts_mt', color = "donor")

In [ ]:
sc.pl.scatter(sample_object_new, x='total_counts', y='n_genes_by_counts', color = "donor")

### Add sample sex covariate

In [ ]:
annot = sc.queries.biomart_annotations(
        "hsapiens",
        ["ensembl_gene_id", "external_gene_name", "start_position", "end_position", "chromosome_name"],
    ).set_index("external_gene_name")

In [ ]:
annot.head()

In [ ]:
chrY_genes = sample_object_new.var_names.intersection(annot.index[annot.chromosome_name == "Y"])
chrY_genes

In [ ]:
sample_object_new.obs['percent_chrY'] = np.sum(
    sample_object_new[:, chrY_genes].X, axis = 1).A1 / np.sum(sample_object_new.X, axis = 1).A1 * 100

In [ ]:
sample_object_new.obs["XIST-counts"] = sample_object_new.X[:,sample_object_new.var_names.str.match('XIST')].toarray()

sc.pl.scatter(sample_object_new, x = 'XIST-counts', y = 'percent_chrY', color = "donor")

In [ ]:
sc.pl.violin(sample_object_new, ["XIST-counts", "percent_chrY"], jitter = 0.4, groupby = 'donor', rotation = 45)

### Calculate cell cycle scores

In [ ]:
!if [ ! -f /Users/mairi.mcclean/cell_cycle_gene.txt ]; then curl -o /Users/mairi.mcclean/cell_cycle_gene.txt https://raw.githubusercontent.com/theislab/scanpy_usage/master/180209_cell_cycle/data/regev_lab_cell_cycle_genes.txt; fi

In [ ]:
cell_cycle_genes = [x.strip() for x in open('/Users/mairi.mcclean/cell_cycle_gene.txt')]
print(len(cell_cycle_genes))

# Split into 2 lists
s_genes = cell_cycle_genes[:43]
g2m_genes = cell_cycle_genes[43:]

cell_cycle_genes = [x for x in cell_cycle_genes if x in sample_object_new.var_names]
print(len(cell_cycle_genes))

- Create basic `anndata` for score calculation

In [ ]:
adata_log = anndata.AnnData(X = sample_object_new.X,  var = sample_object_new.var, obs = sample_object_new.obs)
sc.pp.normalize_total(adata_log, target_sum = 1e6, exclude_highly_expressed = True)
sc.pp.log1p(adata_log)

In [ ]:
sc.tl.score_genes_cell_cycle(adata_log, s_genes = s_genes, g2m_genes = g2m_genes)
sc.pl.violin(adata_log, ['S_score', 'G2M_score'],
             jitter = 0.4, groupby = 'donor', rotation = 45)

In [ ]:
sample_object_new.obs['S_score'] = adata_log.obs['S_score']
sample_object_new.obs['G2M_score'] = adata_log.obs['G2M_score']
sample_object_new

### Predict doublets

In [ ]:
scrub = scr.Scrublet(sample_object_new.X)
sample_object_new.obs['doublet_scores'], sample_object_new.obs['predicted_doublets'] = scrub.scrub_doublets()
scrub.plot_histogram()

sum(sample_object_new.obs['predicted_doublets'])

In [ ]:
#check if our predicted doublets also have more detected genes in general

sc.pl.violin(sample_object_new, ['n_genes_by_counts', 'total_counts', 'pct_counts_mt','pct_counts_ribo', 'predicted_doublets'],
             jitter = 0.2, groupby = 'donor', rotation = 45)

In [ ]:
sc.pl.violin(sample_object_new, ['n_genes_by_counts', 'total_counts', 'pct_counts_mt','pct_counts_ribo', 'predicted_doublets'],
             jitter = 0.2, groupby = 'sample', rotation = 45)

### Prepare counts for individual slots


In [ ]:
sample_object_new.raw = sample_object_new.copy()
sample_object_new.layers['counts'] = sample_object_new.X.copy()
sample_object_new.layers["sqrt_norm"] = np.sqrt(
    sc.pp.normalize_total(sample_object_new, inplace = False)["X"]
)
sample_object_new

### Export object

In [ ]:
sample_object_new.write('/Users/mairi.mcclean/github/data/tb_pbmc_datasets/CaiY2022_TB_QCed_pre-process_mm221122.h5ad')